<a href="https://colab.research.google.com/github/manikcb/sem_project/blob/main/source_code_project_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U "tensorflow-text==2.8.*"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.1/498.1 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.3/462.3 KB 25.6 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd


In [6]:
df = pd.read_csv("/content/drive/MyDrive/dataset/Untitled folder/datatwest2.csv")
df.head(10)

,message,category
0,Hey dumb fuck celebs stop doing something for ...,harass
1,"Drink Bleach Bit “@tayyoung_: FUCK OBAMA, dumb...",harass
2,mmmm #MKR Forget Deconstruction @lisamromano @...,not_harass
3,BLOCK ME THE FUCK BACK YOUR DUMB! Nigga comes ...,harass
4,#Kerala RTE will lead to shutting down of 'Bad...,not_harass
5,I don’t like bragging about really anything I ...,harass
6,Preseason,not_harass
7,this dumb bytch wanna get fuckd up RT @tayyoun...,harass
8,I guess him I ain't a nigger lol RT @CainTheGr...,harass
9,help go back to ur old old old old layout like...,harass


In [8]:
df_harass = df[df['category']=='harass']
df_not_harass = df[df['category']=='not_harass']


In [9]:
df_not_harass_downsampled = df_not_harass.sample(df_harass.shape[0],replace=True)
df_balanced = pd.concat([df_not_harass_downsampled, df_harass])
df_balanced['category'].value_counts()


not_harass    709
harass        709
Name: category, dtype: int64

In [10]:
df_balanced['harass']=df_balanced['category'].apply(lambda x: 1 if x=='harass' else 0)
df_balanced.sample(2)

,message,category,harass
378,RT @mykitchenrules: Nawwww 😭😭😭 #MKR,not_harass,0
588,"New Idea headline: ""#MKR shock! What are they ...",not_harass,0


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_balanced['message'],df_balanced['harass'], stratify=df_balanced['harass'],test_size=0.15)
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)
model = tf.keras.Model(inputs=[text_input], outputs = [l])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [12]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)
model.fit(X_train, y_train, epochs=10)
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

Epoch 1/10
 3/38 [=>............................] - ETA: 11:01 - loss: 0.6689 - accuracy: 0.5833 - precision: 0.5185 - recall: 0.3415

KeyboardInterrupt: ignored